In [2]:
from preparator import Preparator 
import numpy as np 
from solver import Solver 
prep = Preparator()

from preparator import Preparator 
from logger import Logger 
import pandas as pd 
from datetime import datetime
import tqdm 
#Header
# Id (int),
# Valid (int),
# Updated (str),
# Matrix (mat),
# Size (int),
# Type (str),
# Rank (int),
# Density (flt),
# Stiffness (flt),
# Other (flt),
# Test_solver (str),
# Test (flt),
# Test_state (vec),
# Test_iterations (int),
# Test_time (flt),
# Train_solver (str),
# Preprocessing (str),
# Postprocessing (str),
# Train (flt),
# Train_state (vec),
# Train (flt), 
# Train_iterations_1 (int),
# Train_iterations_2 (int),
# Train_iterations_3 (int),
# Train_shots (int),
# Train_cost (flt),
# Train_time (flt),
# Flags (str)
# Baren (int)


In [3]:
# Prepare source dataset 

source_name = 'data/benchmark/Sboost_{sboost}_ansatz_{ansatz}_stepsize_{stepsize}_shots_{shots}.csv'

Config = {
    'size' : 10,
    'count' : 100, 
    'shots' : 1000, 
    'sboost' : 1e8,
    'stepsize' : 0.25, 
    'accepterr': [0.01,0.05,0.01], 
    'iterations_limit' : 100, 
    'baren_err' : 0.01,
    'baren_threshold': 10,
    'ansatz' : 'MA-PIA'
}

template = pd.read_csv('data/template_benchmark.csv')

log = Logger (log_template = template, log_path = source_name.format(size = Config['size'], sboost = Config['sboost'], stepsize = Config['stepsize'], shots = Config['shots'], ansatz = Config['ansatz']))
exists = log.create_logfile()

log_file, id = log._load_logfile()


In [4]:


for i in tqdm.tqdm(range(Config['count']), desc = f"Generating matrices to solve with sboost {Config['sboost']}"):
    
    id += 1
    matrix = np.random.uniform(size = (Config['size'], Config['size']), low = -1, high = 1)
    matrix = (matrix.T + matrix) / 2
    current_datetime = datetime.now()
    data = { 
        'Id (int)' : id, 
        'Valid (int)' : 1, 
        'Updated (str)' : current_datetime.strftime("%Y-%m-%d %H:%M:%S"),
        'Matrix (mat)' : matrix,
        'Size (int)' : Config['size'],
        'Type (str)' : 'Benchmark',
        'Rank (int)' : -1,
        'Density (flt)' : -1.0,
        'Stiffness (flt)' : -1.0, 
        'Penalty (flt)' : -1.0,
        'Sboost (flt)' : Config['sboost'],
        'Test_solver (str)' : '-',
        'Test (flt)' : -1.0,
        'Test_state (vec)' : [],
        'Test_iterations (int)' : -1,
        'Test_time (flt)' : -1.0,
        'Train_solver (str)' : '-',
        'Preprocessing (str)' : '-',
        'Postprocessing (str)' : '-',
        'Train (flt)' : -1.0,
        'Train_state (vec)' : [],
        'Train_iterations (int)' : -1, 
        'Train_iterations_1 (int)' : -1,
        'Train_iterations_2 (int)' : -1,
        'Train_iterations_3 (int)' : -1,
        'Train_shots (int)': -1, 
        'Train_cost (flt)' : -1.0,
        'Train_time (flt)' : -1.0, 
        'Flags (str)' : '',
        'Baren (int)' : -1
    }
    
    log.log(data)


display(log.read(1))

Generating matrices to solve with sboost 100000000.0:  16%|█▌        | 16/100 [00:00<00:00, 157.09it/s]

Generating matrices to solve with sboost 100000000.0: 100%|██████████| 100/100 [00:00<00:00, 110.06it/s]


{'Id (int)': 1,
 'Valid (int)': 1,
 'Updated (str)': '2025-09-01 19:29:09',
 'Matrix (mat)': array([[ 0.33984776,  0.59650208, -0.1313845 ,  0.02527756,  0.54619985,
         -0.17468878,  0.6014892 ,  0.50697242, -0.54844537, -0.13442774],
        [ 0.59650208,  0.10274524, -0.19980245, -0.76346783,  0.37115328,
         -0.14952574,  0.25477122,  0.15227834, -0.35789518, -0.20712598],
        [-0.1313845 , -0.19980245,  0.20562289, -0.62200379,  0.30263077,
          0.37068325, -0.40290254,  0.21045702,  0.30093666, -0.73642429],
        [ 0.02527756, -0.76346783, -0.62200379,  0.31631308,  0.49766738,
          0.023766  , -0.06968882,  0.30389806, -0.06768901,  0.8214402 ],
        [ 0.54619985,  0.37115328,  0.30263077,  0.49766738, -0.93073043,
         -0.56788041, -0.09672308,  0.4594314 ,  0.28633457, -0.80961989],
        [-0.17468878, -0.14952574,  0.37068325,  0.023766  , -0.56788041,
         -0.70894234, -0.05804103,  0.62861839, -0.11262834, -0.20494556],
        [ 0.60

In [5]:
import pennylane as qml 
testdev = qml.device ('default.qubit', wires = Config['size'], shots = Config['shots'])

In [6]:

def test (ids : np.ndarray, solver : str = 'bruteforce', log_data : bool = False, progressbar : bool = False ):

    ''' 
    ids : np.ndarray 
        a list of ids to compute 
    solver : str 
        solver to compute with 
    ''' 
    
    tester = Solver(dev = testdev, ansatz = '')

    if (progressbar):
        id_range = tqdm(ids, desc = f"Testing matrices with ids {min(ids)} <--> {max(ids)}") 
    else: 
        id_range = ids
    processed_data = []
    for id in id_range:

        data = log.read(id)

        print(data)
        def test_solver(Q : np.ndarray, solver : str = 'bruteforce'): 
            ''' 
            tester wrapped
            '''

            match solver: 
                case 'bruteforce':
                    J, h = prep.isingForm(Q)
                    return tester.checkIsing(J, h)
                case 'GW':
                    pass
                case 'Annealing': 
                    pass
            
        Q = data['Matrix (mat)']

        tested_data = data.copy()
        
        test, test_state, test_time = test_solver(Q)

        current_datetime = datetime.now()

        tested_data['Test (flt)'] = test 
        tested_data['Test_state (vec)'] = test_state[0].numpy()
        tested_data['Test_time (flt)'] = test_time
        tested_data['Test_solver (flt)'] = solver
        tested_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
        
        processed_data.append(tested_data)

        if (log_data): 
            log.log(tested_data, byid = True, Id = id)
    return processed_data


In [ ]:
# test(range(1,21), solver = 'bruteforce')

Columns in data/stiffness/Stiffness_size_10_stiffs_[1.0, 10.0, 100.0, 1000.0, 10000.0, 100000.0, 1000000.0]_stepsize_0.25_shots_1000.csv are broken. Expected Index(['Id (int)', 'Valid (int)', 'Updated (str)', 'Matrix (mat)',
       'Size (int)', 'Type (str)', 'Rank (int)', 'Density (flt)',
       'Stiffness (flt)', 'Penalty (flt)', 'Other (flt)', 'Test_solver (str)',
       'Test (flt)', 'Test_state (vec)', 'Test_iterations (int)',
       'Test_time (flt)', 'Train_solver (str)', 'Preprocessing (str)',
       'Postprocessing (str)', 'Train (flt)', 'Train_state (vec)',
       'Train_iterations (int)', 'Train_iterations_1 (int)',
       'Train_iterations_2 (int)', 'Train_iterations_3 (int)',
       'Train_cost (flt)', 'Train_shots (int)', 'Train_time (flt)',
       'Flags (str)', 'Baren (int)'],
      dtype='object') and got Index(['Id (int)', 'Valid (int)', 'Updated (str)', 'Matrix (mat)',
       'Size (int)', 'Type (str)', 'Rank (int)', 'Density (flt)',
       'Stiffness (flt)', 'Other 

KeyboardInterrupt: 

In [7]:


ids = range (1,101)
cores = 25
poolsize = len(ids)//cores

from multiprocessing import Pool
from functools import partial

if (__name__ == '__main__') and (len(ids) % poolsize == 0): 
    
    solvers = [] 

    ids = np.array(ids) 
    index = 0
    pool_ids = ids.reshape(len(ids) // poolsize, poolsize)
    display(pool_ids)
    
    args = []
    for i in range (len(pool_ids)):
        if (Config['size'] > 12):
            args.append((pool_ids[i], 'GW'))
        else: 
            args.append((pool_ids[i], 'bruteforce'))
    #print(args)
    with Pool(cores) as pool:
        processed_data = pool.starmap(test, args)


for i in range(cores):
    for data in processed_data[i]:
        log.log(data = data, byid = True, Id = data['Id (int)'])

# If the error occured you still can try to get processed_data 



array([[  1,   2,   3,   4],
       [  5,   6,   7,   8],
       [  9,  10,  11,  12],
       [ 13,  14,  15,  16],
       [ 17,  18,  19,  20],
       [ 21,  22,  23,  24],
       [ 25,  26,  27,  28],
       [ 29,  30,  31,  32],
       [ 33,  34,  35,  36],
       [ 37,  38,  39,  40],
       [ 41,  42,  43,  44],
       [ 45,  46,  47,  48],
       [ 49,  50,  51,  52],
       [ 53,  54,  55,  56],
       [ 57,  58,  59,  60],
       [ 61,  62,  63,  64],
       [ 65,  66,  67,  68],
       [ 69,  70,  71,  72],
       [ 73,  74,  75,  76],
       [ 77,  78,  79,  80],
       [ 81,  82,  83,  84],
       [ 85,  86,  87,  88],
       [ 89,  90,  91,  92],
       [ 93,  94,  95,  96],
       [ 97,  98,  99, 100]])

{'Id (int)': 33, 'Valid (int)': 1, 'Updated (str)': '2025-09-01 19:29:09', 'Matrix (mat)': array([[-0.8917337 , -0.46892289, -0.44917568,  0.00683325,  0.39088776,
         0.16617984, -0.18051971, -0.60949516,  0.43317774, -0.22508745],
       [-0.46892289, -0.97216968,  0.77853734,  0.45747201,  0.24075655,
         0.16599344, -0.11483141,  0.6944784 , -0.05031788,  0.35374376],
       [-0.44917568,  0.77853734,  0.39595089,  0.36187403,  0.43363389,
        -0.77081029, -0.87550187,  0.17753216,  0.61508924,  0.19220863],
       [ 0.00683325,  0.45747201,  0.36187403, -0.06798419, -0.08255478,
        -0.38692665, -0.48324531,  0.00190694,  0.80537187, -0.70456243],
       [ 0.39088776,  0.24075655,  0.43363389, -0.08255478, -0.95927135,
        -0.56190981, -0.3849158 ,  0.09218617,  0.66877049, -0.09945984],
       [ 0.16617984,  0.16599344, -0.77081029, -0.38692665, -0.56190981,
         0.68285243,  0.06296074, -0.17021939, -0.46502753,  0.44262304],
       [-0.18051971, -0.114

In [ ]:

def train (data : list, trainer : Solver, stepsize: float = 0.1, depth : int = 1, log_data = False): 
    ''' 
    ids : np.ndarray 
        array of ids to train
    solver : str 
        solver with which will be trained 
    preprocessing : str 
    postprocessing : str 
    '''

    #print(f'training ids {ids}')
    #print(data)
    processed_data = []
    for line in data:
        #print(type(line))
        Q = line['Matrix (mat)']

        trained_data = line.copy()
        
        train, train_bitstring, train_state, train_iterations, train_itererr, train_time, baren_flag, baren_index = trainer.solve(Q = Q, depth = depth, sol = line['Test (flt)'], stepsize = stepsize, log = True, sboosted = True, sboost = line['Sboost (flt)'])
        print(f'Trained id = {line['Id (int)']} with train {train}')
        trained_data['Train (flt)'] = train
        trained_data['Train_iterations (int)'] = train_iterations
        trained_data['Train_state (vec)'] = train_bitstring
        for i in range(1, 4): trained_data[f'Train_iterations_{i} (int)'] = train_itererr[i - 1]
        trained_data['Train_time (flt)'] = train_time
        
        current_datetime = datetime.now()
        trained_data['Updated (str)'] = current_datetime.strftime("%Y-%m-%d %H:%M:%S")
        
        if (baren_flag == True):
            trained_data['Flags (str)'] = f'Baren_{baren_index}'
        trained_data['Train_shots (int)'] = trainer.dev.shots.total_shots
        trained_data['Train_solver (str)'] = trainer.ansatz
        trained_data['Preprocessing (str)'] = trainer.preprocessing 
        trained_data['Postprocessing (str)'] = trainer.postprocessing
        if (log_data):
            log.log(trained_data, byid = True, Id = id)
        processed_data.append(trained_data)
    return processed_data

In [4]:
from multiprocessing import Pool
import pandas as pd 
import pennylane as qml 

cores = 25

solvers = [] 

depth = 3

template = pd.read_csv('data/template_benchmark.csv')

for i in range (cores):
    dev = qml.device ('lightning.qubit', wires = Config['size'], shots = Config['shots'])
    solvers.append(Solver(dev = dev, ansatz = Config['ansatz'], gate = 'X', iterations_limit = Config['iterations_limit'], accepterr = Config['accepterr'], baren_check = True, baren_rerr = Config['baren_err']))


log = Logger(log_template = template, log_path = source_name.format(size = Config['size'], sboost = Config['sboost'], stepsize = Config['stepsize'], shots = Config['shots'], ansatz = Config['ansatz']))
train_dataset, _ = log._load_logfile()
#print(train_dataset)

ids = np.array(range(1, 101))
ids = ids.reshape(cores, len(ids) // cores) 

data = [[log.read(id) for id in subset] for subset in ids]

#print(data)
if (__name__ == '__main__') and (len(train_dataset) % cores):
    
    args = [(data[i], solvers[i], Config['stepsize'], depth) for i in range (cores)]
    #print(train_dataset)
    #print(args)
    with Pool(cores) as pool:
        processed_data = pool.starmap(train, args)


for i in range(cores):
    for data in processed_data[i]:
        log.log(data = data, byid = True, Id = data['Id (int)'])




457800.4972576307301763.2258525583357300.9311819763


190908.13292832786181337.7101260554367203.20853725646229672.66423617842855278.5928447846




501037.7349971816537108.35032987352668937.3583617895

411498.1909779651

104952.01867143088
74163.39257270299
89166.67088587453
137559.6538763145577656.1597529243

5255232.434680003
163153.72493856517
187087.85028483727
64976.76775699185
127958.3197927934
207329.6586281158
209500.26158830494260056.5121154215

Ansatz: MA-PIA Iteration: 1 | State: [ 0.082  0.186 -0.026 -0.076  0.422  0.358 -0.132 -0.072  0.046 -0.05 ] | Sol: -2.573686868416586 | Cursol: -1.6557354935244848 | Curenergy: -0.46087521932314546 | Preenergy: 0 | Baren_queue: [] | 100000000.0
Ansatz: MA-PIA Iteration: 1 | State: [-0.668  0.088  0.558 -0.496  0.014  0.77   0.352  0.242  0.254 -0.388] | Sol: -3.8180051872350544 | Cursol: -1.4972605852431284 | Curenergy: -0.6918736842074558 | Preenergy: 0 | Baren_queue: [] | 100000000.0
Ansatz: MA-PIA Iteration: 1 | State: [ 0.04  -0.17

Process ForkPoolWorker-18:
Process ForkPoolWorker-23:
Process ForkPoolWorker-17:
Process ForkPoolWorker-19:
Process ForkPoolWorker-12:
Process ForkPoolWorker-14:
Process ForkPoolWorker-22:
Process ForkPoolWorker-20:
Process ForkPoolWorker-10:
Process ForkPoolWorker-15:
Process ForkPoolWorker-24:
Process ForkPoolWorker-16:
Process ForkPoolWorker-21:
Process ForkPoolWorker-13:


KeyboardInterrupt: 

In [ ]:
display(processed_data[3]['Id (int)'])
display(processed_data[3]['Train (flt)'])

arr = [1,2,3,4,5,6,7,8,9,10]
arr = np.array(arr)
arr = arr.reshape(len(arr) // 2, 2)

print(arr)
train(range(1,21), trainer = solvers[0], progressbar = True)
